In [1]:
#Python Imports
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
import re
from pandas.io.json import json_normalize
import json
import pandas as pd
import numpy as np
from surprise import SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import NMF
from pandas.io.json import json_normalize
from pymongo import MongoClient
from collections import defaultdict
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import re
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
def convert_ids(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('int64')

def convert_to_float(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('float64')

def to_json(csv_entry):
    return json.loads(re.sub('\'', '"', csv_entry))


def get_top_n(predictions, n=10):
    '''COPIED FROM SUPRISE API
    Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_movie_name(movie_id):
    return ratings_with_movie_names[ratings_with_movie_names.id == movie_id]['original_title'].iloc[0]


def print_user_prediction(userId, predictions_dict, meta_df):
    users_viewed_movies = ratings_with_movie_names[ratings_with_movie_names['userId'] == userId][
        ['rating', 'original_title']]
    print(f'User {userId} has viewed the following movies:\n')

    for row in users_viewed_movies.itertuples():
        rating = row[1]
        original_title = row[2]
        print(f'\t{original_title}, Rating: {rating}')

    print(f'\nThe following movies are recommended for User {userId}\n')
    recommended_movies = [get_movie_name(mov_id[0], meta_df) for mov_id in predictions_dict[userId]]

    for movie in recommended_movies:
        print(f'\t{movie}')
        
def clean(text, tokenizer, stemmer):
    """
    Cleans Text with Regexes
    :param text:
    :return: text:
    """
    doc = ''.join(text).lower()
    doc = re.sub(r'[<>\{}/;|\[\]-]', ' ', doc)
    doc = re.sub(r'[0-9]', ' ', doc)
    doc = re.sub(r'\'', ' ', doc)
    doc = re.sub(r'=', ' ', doc)
    doc = re.sub(r':', ' ', doc)
    doc = re.sub(r'"', ' ', doc)
    doc = re.sub(r'\s+', ' ', doc)
    doc = re.sub(r'\(', ' ', doc)
    doc = re.sub(r'\)', ' ', doc)
    doc = re.sub(r'\s{2,}', ' ', doc)
    doc = re.sub(r'\.', '', doc)
    doc = re.sub(r',', '', doc)

    return doc

def get_movie_name(movie_id, movie_meta_df):
    return movie_meta_df[movie_meta_df.id == movie_id]['original_title'].iloc[0]

def get_movie_id(title, movie_meta_df):
    return movie_meta_df[movie_meta_df.title == title]['id'].iloc[0]


def get_all_movies_in_cluster(cluster_number, cluster_dict, meta_df):
    movies = cluster_dict[cluster_number]
    return [get_movie_name(mov, meta_df) for mov in movies]

####

# cluster_distribution = [len(movies) for (clust, movies) in movie_summaries_clustered.items()]

def get_cluster_number(movie, cluster_zip):
    for cluster, movie_id in cluster_zip:

        if movie_id == movie:
            return cluster

    raise Exception('Movie not found in cluster')

In [3]:
movies_metadata_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                                 , converters={ 'id': lambda x: convert_ids(x)
                                               , 'imdb_id': lambda x: convert_ids(x)
                                               ,'popularity': lambda x: convert_to_float(x)
                                               ,'genres': lambda x: to_json(x)}
                                 , usecols=['id', 'original_title'
                                                , 'genres', 'homepage'
                                                , 'overview', 'popularity', 'poster_path'
                                                , 'release_date', 'revenue', 'runtime'
                                                , 'spoken_languages', 'tagline', 'title'
                                                , 'vote_average', 'vote_count']
                                , dtype={'populariy': np.float64}
                                , parse_dates=True)


movies_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'original_title', 'belongs_to_collection'
                                 , 'budget', 'genres', 'homepage'
                                 ,'imdb_id', 'overview', 'popularity', 'poster_path'
                                 , 'production_companies','release_date', 'revenue', 'runtime',
                                 'spoken_languages', 'status', 'tagline', 'title', 'video',
                                 'vote_average', 'vote_count'])

ratings_df = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')



content_filter_df = pd.read_pickle('../data/content_filter_df.pkl')
content_filter_df = content_filter_df[['id',
 'popularity',
 #'release_date',
 'vote_average',
 'release_year',
 0,1,2,3,4,5, 6, 7, 8,9,10,11,12,13,14,15,16,17,18,19]]
content_filter_df = content_filter_df.dropna()

idx = pd.Index(content_filter_df['id'])
idx
content_filter_df.index = idx

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## SVD

In [4]:
###May need Fuzzy matching, but for now:
movies_df = movies_df[movies_df.spoken_languages == """[{'iso_639_1': 'en', 'name': 'English'}]"""]

In [5]:
ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')
ratings_with_movie_names = ratings_with_movie_names[ratings_with_movie_names.original_title.isnull() == False]

#### Create Model

In [6]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_with_movie_names[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

##### You can try different Algorithms here

In [7]:
algo = SVD(verbose=True)
algo.fit(trainset)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8961  0.8953  0.8911  0.8952  0.9080  0.8971  0.0057  
MAE (testset)     0.6947  0.6978  0.6861  0.6885  0.6960  0.6926  0.0045  
Fit time          0.91    0.93    0.93    0.89    0.83    0.90    0.04    
Test time         0.03    0.03    0.03    0.02    0.02    0.03    0.00    


{'test_rmse': array([0.89611314, 0.89527363, 0.89107297, 0.89519268, 0.90797342]),
 'test_mae': array([0.69467971, 0.69782441, 0.68613559, 0.68847842, 0.69597631]),
 'fit_time': (0.9071729183197021,
  0.9258358478546143,
  0.9292879104614258,
  0.8936519622802734,
  0.8306610584259033),
 'test_time': (0.026726961135864258,
  0.027556896209716797,
  0.028174161911010742,
  0.023527145385742188,
  0.023064851760864258)}

In [14]:
predictions = algo.test(testset)
### Tune this value to get fewer results faster, but less options to choose from
top_n = get_top_n(predictions, 5)


predicted_movies_by_name = defaultdict(list)

### This builds the dictionary of predicted movies for all users
for key, value in top_n.items():
    predicted_movies_by_name[key] = [get_movie_name(mov_id[0], movies_metadata_df) for mov_id in value]

**At this point, you should have user personas to get a pool of movies to choose from, not simply pre-made users:**


In [17]:
print_user_prediction(321, top_n, movies_metadata_df)

User 321 has viewed the following movies:

	The Endless Summer, Rating: 5.0
	Rumble Fish, Rating: 5.0
	A River Runs Through It, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 5.0
	The Poseidon Adventure, Rating: 5.0
	Apollo 13, Rating: 4.0
	Men in Black II, Rating: 4.0
	A View to a Kill, Rating: 4.0
	Videodrome, Rating: 4.0
	The Thomas Crown Affair, Rating: 5.0
	Hollywoodland, Rating: 4.0
	Bridge to Terabithia, Rating: 4.0
	Rope, Rating: 3.0
	A Time to Kill, Rating: 4.0
	Little Buddha, Rating: 3.0
	Jungle Fever, Rating: 4.0
	Return of the Jedi, Rating: 2.0
	Don Juan DeMarco, Rating: 2.0
	Twin Peaks: Fire Walk with Me, Rating: 2.0
	My Name Is Bruce, Rating: 2.0
	The Bachelor, Rating: 3.0
	Cold Mountain, Rating: 4.0
	In the Name of the King: A Dungeon Siege Tale, Rating: 2.0
	Short Circuit, Rating: 1.0

The following movies are recommended for User 321

	Lonely Hearts
	Sleepless in Seattle
	Birdman of Alcatraz
	Within the Woods
	The Million Dollar Hotel


In [18]:
print_user_prediction(47, top_n, movies_metadata_df)

User 47 has viewed the following movies:

	48 Hrs., Rating: 4.0
	Back to the Future Part II, Rating: 4.0
	20,000 Leagues Under the Sea, Rating: 5.0
	Muriel's Wedding, Rating: 4.0
	High Noon, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 4.0
	Faster, Pussycat! Kill! Kill!, Rating: 4.0
	Silent Hill, Rating: 4.0
	The Conversation, Rating: 3.0
	To Kill a Mockingbird, Rating: 3.0

The following movies are recommended for User 47

	Sleepless in Seattle
	The Good Thief
	The Thomas Crown Affair
	Nell
	While You Were Sleeping


In [19]:
print_user_prediction(321, top_n, movies_metadata_df)

User 321 has viewed the following movies:

	The Endless Summer, Rating: 5.0
	Rumble Fish, Rating: 5.0
	A River Runs Through It, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 5.0
	The Poseidon Adventure, Rating: 5.0
	Apollo 13, Rating: 4.0
	Men in Black II, Rating: 4.0
	A View to a Kill, Rating: 4.0
	Videodrome, Rating: 4.0
	The Thomas Crown Affair, Rating: 5.0
	Hollywoodland, Rating: 4.0
	Bridge to Terabithia, Rating: 4.0
	Rope, Rating: 3.0
	A Time to Kill, Rating: 4.0
	Little Buddha, Rating: 3.0
	Jungle Fever, Rating: 4.0
	Return of the Jedi, Rating: 2.0
	Don Juan DeMarco, Rating: 2.0
	Twin Peaks: Fire Walk with Me, Rating: 2.0
	My Name Is Bruce, Rating: 2.0
	The Bachelor, Rating: 3.0
	Cold Mountain, Rating: 4.0
	In the Name of the King: A Dungeon Siege Tale, Rating: 2.0
	Short Circuit, Rating: 1.0

The following movies are recommended for User 321

	Lonely Hearts
	Sleepless in Seattle
	Birdman of Alcatraz
	Within the Woods
	The Million Dollar Hotel


## NLP Topic Modeling Movie Summaries

In [ ]:
nlp_pp = nlp_preprocessor(TfidfVectorizer(stop_words='english'), cleaning_function=clean)
corpus = ' '
sentences = [str(sentence) for sentence in movies_metadata_df.overview.tolist()]

In [ ]:
# Vectorize i.e: format the corpus into a matrix in preparation for Singular Value Decomposition
cv_tfidf = TfidfVectorizer(min_df=5, stop_words='english')  # ngram_range=(1,2)
X_tfidf = cv_tfidf.fit_transform(sentences).toarray()
tfidf_model_df = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names())
# tfidf_model_df = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names())

In [ ]:
# Components is the Amount of dimensionaly-reduction from the corpus of words. This, with cluster-size\
# affects the model performance are the hyperparameters.
svd = TruncatedSVD(n_components=20)
X = svd.fit_transform(tfidf_model_df)

# print(svd.explained_variance_ratio_.sum())

In [ ]:
km = KMeans(n_clusters=60, init='k-means++', max_iter=100, n_init=1, verbose=True)
km.fit(X)

In [ ]:
#### Find the cluster of a movie by movie id
clustered = zip(km.labels_, movies_metadata_df['id'])  # zip(tfidf_model_df.index.values, km.labels_)
print(get_cluster_number(862, clustered))


####

## Explore the Cluster Results

clustered = zip(km.labels_, movies_metadata_df['id'])  # zip(tfidf_model_df.index.values, km.labels_)

#### Needs Functionalization/Abstraction
from collections import defaultdict

movie_summaries_clustered = defaultdict(list)

for cluster, movie_id in clustered:
    movie_summaries_clustered[cluster].append(movie_id)


#### List all the movies in a cluster
print(get_all_movies_in_cluster(20, movie_summaries_clustered))
####

## Cosine Similarity for Content Filter

In [37]:
# Given a matrix, return 10 movie tuples with the most similarity (id, name)
from collections import namedtuple

CosineSimilarity = namedtuple('CosineSimilarity', ['id1', 'id2', 'similarity', 'name1', 'name2'])

In [111]:
def top_n_closest_content_filtered(n, movie_id, content_filter_df):
    similarity_arr = cosine_similarity(np.array(content_filter_df.loc[movie_id]).reshape(1, -1), content_filter_df)[0].tolist()
    tups = [(similarity_arr.index(value), value) for value in similarity_arr]
    tups = sorted(tups, key=lambda x: x[1], reverse=True)
    return tups[0:n]
    

In [134]:
sleepless_filtered_recs = top_n_closest_content_filtered(100, 858, content_filter_df)
sleepless_filtered_recs

[(534, 0.9999999999999998),
 (1904, 0.9999985798898556),
 (1208, 0.9999981736066476),
 (4137, 0.9999979433200008),
 (2681, 0.9999972682954646),
 (2792, 0.9999971992504212),
 (362, 0.999996301872156),
 (8232, 0.999996033283064),
 (984, 0.9999959498257401),
 (10833, 0.9999958404012185),
 (4870, 0.9999958239470732),
 (6816, 0.999995472066687),
 (2863, 0.9999946890182448),
 (2070, 0.9999937776328239),
 (4086, 0.9999934671526973),
 (2989, 0.999993301817415),
 (2077, 0.9999926890765405),
 (4899, 0.9999924966707522),
 (6451, 0.999992266144631),
 (535, 0.9999920723155129),
 (3567, 0.9999915709936217),
 (7692, 0.9999900927060298),
 (8659, 0.9999898587099378),
 (4504, 0.9999892343229313),
 (3343, 0.9999883011996182),
 (3237, 0.9999873285484169),
 (1201, 0.9999859683288698),
 (1248, 0.999984520971692),
 (1912, 0.9999844888845639),
 (7761, 0.9999840724417758),
 (0, 0.9999840666091478),
 (1905, 0.9999838348133764),
 (14377, 0.9999823684434473),
 (1793, 0.9999814882134689),
 (2803, 0.999979896600553

In [135]:
videodrome_recs = top_n_closest_content_filtered(100, 837, content_filter_df)
videodrome_recs

[(6614, 0.9999999999999998),
 (1198, 0.9999995097028028),
 (4008, 0.9999991389263577),
 (9200, 0.9999989572521445),
 (6953, 0.9999986910447686),
 (1905, 0.9999980671834254),
 (9537, 0.9999976013364797),
 (1248, 0.9999975103862115),
 (1201, 0.9999971499357722),
 (1793, 0.9999969306607781),
 (3237, 0.9999965440703428),
 (3343, 0.9999957554021307),
 (1200, 0.9999957552508402),
 (6617, 0.9999950239778762),
 (7761, 0.999993762261875),
 (2077, 0.9999936099151473),
 (8659, 0.9999932524968942),
 (12978, 0.9999927727129296),
 (4086, 0.9999924818697025),
 (6451, 0.999991177576172),
 (4870, 0.9999893304264468),
 (14377, 0.9999886797873373),
 (4137, 0.9999859749819323),
 (4899, 0.999984350412921),
 (2681, 0.9999838953145571),
 (10698, 0.9999826508811969),
 (362, 0.999981562905407),
 (534, 0.999973728971039),
 (1208, 0.9999688226084048),
 (2322, 0.9999661617492284),
 (2863, 0.9999651504414468),
 (1912, 0.9999626845604357),
 (1904, 0.9999624868507314),
 (7625, 0.9999601337332559),
 (882, 0.999958196

In [136]:
top_filtered_to_movie_rec_tuples(videodrome_recs, 837, movies_metadata_df)

lookup failed 6614
lookup failed 1198
lookup failed 4008
lookup failed 9200
lookup failed 6953
lookup failed 1905
lookup failed 9537
lookup failed 1201
lookup failed 3237
lookup failed 3343
lookup failed 1200
lookup failed 6617
lookup failed 7761
lookup failed 8659
lookup failed 12978
lookup failed 4086
lookup failed 6451
lookup failed 4870
lookup failed 14377
lookup failed 4137
lookup failed 4899
lookup failed 2681
lookup failed 10698
lookup failed 362
lookup failed 1208
lookup failed 2863
lookup failed 7625
lookup failed 882
lookup failed 3259
lookup failed 8232
lookup failed 2792
lookup failed 5266
lookup failed 6816
lookup failed 2070
lookup failed 0
lookup failed 7692
lookup failed 4504
lookup failed 3567
lookup failed 2606
lookup failed 5337
lookup failed 2560
lookup failed 33804
lookup failed 25819
lookup failed 32106
lookup failed 2803
lookup failed 7462
lookup failed 7163
lookup failed 6822
lookup failed 46
lookup failed 7282
lookup failed 7706
lookup failed 3320
lookup failed

[MovieRecommendation(id_origin=837, title='Videodrome', id_2=1248, title_2='Hannibal Rising', similarity=0.9999975103862115),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=1793, title_2='Max Dugan Returns', similarity=0.9999969306607781),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=2077, title_2='The Rink', similarity=0.9999936099151473),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=534, title_2='Terminator Salvation', similarity=0.999973728971039),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=2322, title_2='Sneakers', similarity=0.9999661617492284),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=1912, title_2='Free Rainer - Dein Fernseher lügt', similarity=0.9999626845604357),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=1904, title_2='Memoirs of a Geisha', similarity=0.9999624868507314),
 MovieRecommendation(id_origin=837, title='Videodrome', id_2=4170, title_2='Trade', similarity=0.99995785

In [127]:
from collections import namedtuple

CosineSimilarity = namedtuple('CosineSimilarity', ['id1', 'id2', 'similarity', 'name1', 'name2'])

MovieRecommendation = namedtuple('MovieRecommendation', ['id_origin', 'title'
                                 ,'id_2', 'title_2'
                                ,'similarity'])

def top_filtered_to_movie_rec_tuples(content_filtered_rec_list, origin_movie_id, movie_meta_data_df):
    
    recs = []
    origin_film_name = get_movie_name(origin_movie_id, movie_meta_data_df)
    
    for rec in content_filtered_rec_list:
        try:

            film_2_id = rec[0]
            film_2_name = get_movie_name(rec[0], movie_meta_data_df)
            sim_score = rec[1]

            recs.append(MovieRecommendation(origin_movie_id, origin_film_name
                                                   , film_2_id, film_2_name 
                                                    , sim_score))
        except:
            print(f'lookup failed {rec[0]}')
            pass
    
    return recs
    


In [137]:
top_filtered_to_movie_rec_tuples(sleepless_filtered_recs, 858, movies_metadata_df)

lookup failed 1208
lookup failed 4137
lookup failed 2681
lookup failed 2792
lookup failed 362
lookup failed 8232
lookup failed 4870
lookup failed 6816
lookup failed 2863
lookup failed 2070
lookup failed 4086
lookup failed 4899
lookup failed 6451
lookup failed 3567
lookup failed 7692
lookup failed 8659
lookup failed 4504
lookup failed 3343
lookup failed 3237
lookup failed 1201
lookup failed 7761
lookup failed 0
lookup failed 1905
lookup failed 14377
lookup failed 2803
lookup failed 7462
lookup failed 6953
lookup failed 1198
lookup failed 6614
lookup failed 4008
lookup failed 9200
lookup failed 9537
lookup failed 7282
lookup failed 1200
lookup failed 3360
lookup failed 12978
lookup failed 6617
lookup failed 2408
lookup failed 3434
lookup failed 10698
lookup failed 1128
lookup failed 3259
lookup failed 882
lookup failed 7625
lookup failed 5266
lookup failed 6627
lookup failed 7794
lookup failed 16712
lookup failed 2606
lookup failed 2560
lookup failed 5337
lookup failed 33804
lookup faile

[MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=534, title_2='Terminator Salvation', similarity=0.9999999999999998),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=1904, title_2='Memoirs of a Geisha', similarity=0.9999985798898556),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=984, title_2='Dirty Harry', similarity=0.9999959498257401),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=10833, title_2='Heartbreakers', similarity=0.9999958404012185),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=2989, title_2='Roar', similarity=0.999993301817415),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=2077, title_2='The Rink', similarity=0.9999926890765405),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=535, title_2='Flashdance', similarity=0.9999920723155129),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=

## Model Hybridization

In [3]:
#Model Fitting
#Surprise SVD Collaborative Filtering
#Topic Modeling Based on Movie Descriptions
#ContentFiltering ... Not finished...? Or was it?

In [4]:
#User Creation

In [5]:
#Model Merging
#Genre Constrictions
#Neo4j Graph Degree Restrictions

In [ ]:
class hyprid_recommender():
    
    def __init__(surprise_model, topic_model, content_matrix, neo4j_session):
        
        self.surprise_model = surprise_model
        self.toppic_model = topic_model
        self.content_matrix = content_matrix
        self.neo4j_session = neo4j_session
        
    def recommend(user_profile, last_watched=None):
        """
        Gets a list of movies recommended from the Surprise collaborative filter. 
        This can be any number of films.
        
        I can use the graph at that point to filter the surprise list off of movies that are greater than one-degree.
        
        I can use the content Similarity matrix to rank the movies that are closes to the supplied movies.
        
        user_profile: dictionary containing movies seen and rated. """
        
        movie = 'Top Gun'
        return movie